In [ ]:
import time
print(time.ctime())
%load_ext autoreload
%autoreload 2
%autosave 10


#  Create tardis data file

https://tardis-sn.github.io/tardis/io/configuration/components/models/index.html#id7

```
model:
    structure:
        type: file
        filename: density.dat
        filetype: simple_ascii
        v_inner_boundary: 11000 km/s
        v_outer_boundary: 20000 km/s
   abundances:
        type: file
        filename: abund.dat
        filetype: simple_ascii
```        

In [ ]:
# for plottong
# import matplotlib
# matplotlib.use('TkAgg')
%matplotlib inline
# %matplotlib notebook
from matplotlib import rcParams
# from matplotlib.lines import lineStyles
import matplotlib.pyplot as plt
from matplotlib import gridspec, colors
from matplotlib.patches import Rectangle
# import matplotlib.lines as mlines

if True:
    # plt.style.use('../../lib/mpl/mtl2smv2.stl')
    plt.style.use('./mtl2smv2.stl')
    
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['font.size'] = 14    

import logging

mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.ERROR)


In [ ]:
import os
import sys
import operator
import itertools

import numpy as np
# import scipy
# from scipy import stats
# from scipy import interpolate
# from scipy import optimize
import pandas as pd
# import seaborn as sns
from astropy import constants as consts
from astropy import units as u


sys.path.append(os.path.expanduser('/home/bakl/Sn/Release/python/pystella'))
import pystella as ps


In [ ]:
def fig_save(fig, fname, ext=".pdf"):
    if not fname.endswith(ext):
        fname += ext
    fsave = os.path.expanduser(fname)
    print("Save plot to %s " % fsave)
    fig.savefig(fsave, bbox_inches='tight')
    
figsave = fig_save

### Read STELLA data

Example: https://tardis-sn.github.io/tardis/io/configuration/components/models/converters/stella_to_tardis.html

### Parameters

In [ ]:
path_mdl = './'  #

sn_mname = 'pyrefE5R50M26Ni2m2b5m3Z01'
  
print(f'sn_mname= {sn_mname}')

fname_density_out = 'density.csv'
fname_abund_out = 'abund.csv'
print(f'{fname_density_out=}')
print(f'{fname_abund_out=}')

In [ ]:
models_info =  ps.ls.short(path_mdl)
models_data = ps.ls.long(models_info, path=path_mdl)
print(len(models_data))

In [ ]:
import h5py

def printall(name, obj):
    print(name, dict(obj.attrs))

    
fname = os.path.expanduser( os.path.join(path_mdl, sn_mname+'.h5') )

print(f'{fname=}')
    
with h5py.File(fname,'r') as hf:
    hf.visititems(printall)

In [ ]:
def h5_gr_print(gr, lvl=0):
    #Checkout what keys are inside that group.
    space = "  " * lvl
    lvl += 1
    for key in gr.keys():        
        print("{}{}".format(space, key))
        item = gr[key]
        if isinstance(item, h5py.Group):
            h5_gr_print(item, lvl)
        elif isinstance(item, h5py.Dataset):
            print("{}{} {}".format(space, key, item.shape)) 
#         else:
#             print("{}{} {}".format(space, key, type(item)))

h5data = {}

with h5py.File(fname, "r") as f:
    h5_gr_print(f)

In [ ]:
# fname = os.path.expanduser( os.path.join('~', sn_mname+'.h5') )
h5 = ps.H5Stella(fname)
print(fname)

hyd = h5.Hyd
hyd.Info()
print(hyd.Ntime, hyd.Nzon, hyd.Nvars, hyd.Shape, hyd.Attrs)
print(f'{hyd.R.shape=}    {hyd.T.shape=}')
t_hyd = hyd.T
print('Hyd columns: ',hyd.Columns, ' time: ', len(t_hyd))
print(f'{t_hyd=}')

In [ ]:
# nums = [0, 2, 5, 10, 20, 50]
# columns = hyd.Attrs['columns'].split()
# plt.figure(figsize=(12,12))
# for i, var in enumerate(columns):
#     for num in nums:
#         plt.subplot(2,2,i+1)
#         x = hyd.V[num, :, 0]
#         y = hyd.V[num, :, i]
#         plt.plot(x, y, 'o-', label='{} t={:.3f}'.format(str(var,'utf-8'), hyd.T[num]))
#         plt.xscale('log')
#         plt.yscale('log')
#     plt.legend()

In [ ]:

t_cur = 10  #time[0]
for i, t in enumerate(t_hyd):
    t_idx = i
    if t > t_cur:
        break
        
# t_idx = 10
t = t_hyd[t_idx]
print(f'{t_idx=}   {t=}')


In [ ]:
Ry = hyd.R[t_idx]  # cm
Vy = hyd.V[t_idx]  # 1000 km/s
Rhoy = hyd.Rho[t_idx]  

# Normalization
Vy = 1000 * Vy # km/s


In [ ]:
# data = stella.read_stella_data('mesa.stella.dat')


In [ ]:
# extract outer radius boundaries
# radii = data.loc[:,'outer_edge_r'].values * u.cm
radii = Ry * u.cm

# calculate t_explosion
t_explosion = t * u.day #days

# # calculate outer velocities of each zone assuming homologous expansion (v=r/t)
# velocities = (radii/t_explosion).to(u.km/u.s)

velocities = Vy * 1000.*u.km/u.s

# create a new column of our velocities (OPTIONAL)
data = {'velocity':  velocities,
        'density': Rhoy,
        }

df = pd.DataFrame(data)
df

In [ ]:
density_dat = df[['velocity', 'density']].reset_index(drop=True)
density_dat.to_csv(fname_density_out, sep = ' ')


In [ ]:
eve_elements = ("Ni56", "H", "He", "C", "N", "O", "Ne", "Na", "Mg", "Al"
                , "Si", "S", "Ar", "Ca", "Fe", "Ni")
abun = h5.Yabun

print(abun.shape)
data_abun = {nm: abun[i] for i, nm in enumerate(eve_elements)}
df_abun = pd.DataFrame(data_abun)
df_abun

In [ ]:
# import all elements and isotopes and export to TARDIS
# data_elements = data.iloc[:,12:33].reset_index(drop=True)
df_abun.to_csv(fname_abund_out, sep = ' ')